In [28]:
import cv2 
import pytesseract
import numpy as np
import json

pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'

### Functions for image preprocessing

In [29]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

In [30]:
img = cv2.imread('../receipts/receipt3.jpg')

gray = get_grayscale(img)
thresh = thresholding(gray)

# TODO: are there more useful image preprocessing methods 

img = thresh

In [31]:
image_text = pytesseract.image_to_string(img, lang='deu')
image_text

'MIGROS\n\nGENOSSENSCHAFT MIGROS LUZERN\nM Schlossberg Luzern\n\nArtikelbezeichnung Menge Preis Gespart Total #\nKassentragtasche 1 0.40 0.402\nKinder Mix Kalender 1 11.60 11.60 2\nMonini Classico 11 1 18.50 18.50 1\nCrocchini Rosmarino 1 2.80 2.80 1\nMClass Löffelbiscuit 2 1.00 2.00 1\nZweifel Chips Nature 1 5.95 5.95 1\nServietten 33x33cm 1 4.50 4.50 2\nvalflora vollrahm unt 1 3.50 3.50 1\nvalflora vollrahm unt 1 2.10 2.10 1\nEier Freilandhaltung 1 3.95 3.95 1\nvalflora vollmilch 11 1 1.65 1.65 1\nOatly Haferdr.Barista 1 3.40 3.40 1\nAgnesi Spaghetti N. 3 4 2.50 10.00 1\nMClass Penne 4 1.90 7.60 1\nMClass Skyr Heidelbeer 2 1.25 2.50 1\nYOU Skyr Mango-Passion 1 1.80 1.80 1\nGalbanı Mascarpone 1 6.95 6.95 1\nThomy Tomatenpüree 1 3.40 3.40 1\nZwiebeln 0.281 1.60 0.45 1\nBananen 0.820 2.50 2.05 1\nTotal CHF 95.10\nMastercard 95.10\n\nBUCHUNG Mastercard\n\nKRXXXKKXXKXX2532\n\n30.11.2024 14:08\n\n#31454544*00295699/633131/NA#\n\nTotal EFT CHF: 95.10\n2-2 tel:00295699/633131\nCumulus-Nummer

In [32]:
print(image_text)

MIGROS

GENOSSENSCHAFT MIGROS LUZERN
M Schlossberg Luzern

Artikelbezeichnung Menge Preis Gespart Total #
Kassentragtasche 1 0.40 0.402
Kinder Mix Kalender 1 11.60 11.60 2
Monini Classico 11 1 18.50 18.50 1
Crocchini Rosmarino 1 2.80 2.80 1
MClass Löffelbiscuit 2 1.00 2.00 1
Zweifel Chips Nature 1 5.95 5.95 1
Servietten 33x33cm 1 4.50 4.50 2
valflora vollrahm unt 1 3.50 3.50 1
valflora vollrahm unt 1 2.10 2.10 1
Eier Freilandhaltung 1 3.95 3.95 1
valflora vollmilch 11 1 1.65 1.65 1
Oatly Haferdr.Barista 1 3.40 3.40 1
Agnesi Spaghetti N. 3 4 2.50 10.00 1
MClass Penne 4 1.90 7.60 1
MClass Skyr Heidelbeer 2 1.25 2.50 1
YOU Skyr Mango-Passion 1 1.80 1.80 1
Galbanı Mascarpone 1 6.95 6.95 1
Thomy Tomatenpüree 1 3.40 3.40 1
Zwiebeln 0.281 1.60 0.45 1
Bananen 0.820 2.50 2.05 1
Total CHF 95.10
Mastercard 95.10

BUCHUNG Mastercard

KRXXXKKXXKXX2532

30.11.2024 14:08

#31454544*00295699/633131/NA#

Total EFT CHF: 95.10
2-2 tel:00295699/633131
Cumulus-Nummer .603.400.063
Punktestand per 09.11.2024

In [33]:
entities_1 = [
    'Kassentragtasche 1 0.40 0.402',
    'Kinder Mix Kalender 1 11.60 11.60 2',
    'Monini Classico 11 1 18.50 18.50 1',
    'Crocchini Rosmarino 1 2.80 2.80 1',
    'MClass Löffelbiscuit 2 1.00 2.00 1',
    'Zweifel Chips Nature 1 5.95 5.95 1',
    'Servietten 33x33cm 1 4.50 4.50 2',
    'valflora vollrahm unt 1 3.50 3.50 1',
    'valflora vollrahm unt 1 2.10 2.10 1',
    'Eier Freilandhaltung 1 3.95 3.95 1',
    'valflora vollmilch 11 1 1.65 1.65 1',
    'Oatly Haferdr.Barista 1 3.40 3.40 1',
    'Agnesi Spaghetti N. 3 4 2.50 10.00 1',
    'MClass Penne 4 1.90 7.60 1',
    'MClass Skyr Heidelbeer 2 1.25 2.50 1',
    'YOU Skyr Mango-Passion 1 1.80 1.80 1',
    'Galbanı Mascarpone 1 6.95 6.95 1',
    'Thomy Tomatenpüree 1 3.40 3.40 1',
    'Zwiebeln 0.281 1.60 0.45 1',
    'Bananen 0.820 2.50 2.05 1'
]

entities = []
for entity in entities_1:
    starting_index = image_text.index(entity)
    end_index = starting_index + len(entity)
    entities.append([starting_index, end_index, "RECEIPT_ITEM"])
    print(f'Entity: "{entity}" starts at: {starting_index}, ends at: {end_index}')
    


Entity: "Kassentragtasche 1 0.40 0.402" starts at: 106, ends at: 135
Entity: "Kinder Mix Kalender 1 11.60 11.60 2" starts at: 136, ends at: 171
Entity: "Monini Classico 11 1 18.50 18.50 1" starts at: 172, ends at: 206
Entity: "Crocchini Rosmarino 1 2.80 2.80 1" starts at: 207, ends at: 240
Entity: "MClass Löffelbiscuit 2 1.00 2.00 1" starts at: 241, ends at: 275
Entity: "Zweifel Chips Nature 1 5.95 5.95 1" starts at: 276, ends at: 310
Entity: "Servietten 33x33cm 1 4.50 4.50 2" starts at: 311, ends at: 343
Entity: "valflora vollrahm unt 1 3.50 3.50 1" starts at: 344, ends at: 379
Entity: "valflora vollrahm unt 1 2.10 2.10 1" starts at: 380, ends at: 415
Entity: "Eier Freilandhaltung 1 3.95 3.95 1" starts at: 416, ends at: 450
Entity: "valflora vollmilch 11 1 1.65 1.65 1" starts at: 451, ends at: 486
Entity: "Oatly Haferdr.Barista 1 3.40 3.40 1" starts at: 487, ends at: 522
Entity: "Agnesi Spaghetti N. 3 4 2.50 10.00 1" starts at: 523, ends at: 559
Entity: "MClass Penne 4 1.90 7.60 1" st

In [ ]:
# some JSON
try:
    # TODO: use different file when starting new labling session and update bottom write execution
    file = open('../spacy-ner/training/training_data_session1', 'r')
except FileNotFoundError:
    data = []
else:
    with file:
        data = json.load(file)
    
# a Python object (dict):
x = {
  "text": image_text,
  "entities": entities,
}

data.append(x)

# convert into JSON:
y = json.dumps(data)

with open("../spacy-ner/training/training_data_session1.json", "w") as outfile:
    outfile.write(y)